In [26]:
import sys
import numpy as np
sys.path.insert(0, r'../')
import src.dataset as ds
import src.html_lastfm_parser as hlp
import tensorflow as tf

In [27]:
def create_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))
 
def create_biases(size):
    return tf.Variable(tf.constant(0.05, shape=[size]))

def create_convolutional_layer(input, num_input_channels, conv_filter_size, num_filters):  
    ## We shall define the weights that will be trained using create_weights function.
    weights = create_weights(shape=[conv_filter_size, conv_filter_size, num_input_channels, num_filters])
    ## We create biases using the create_biases function. These are also trained.
    biases = create_biases(num_filters)
 
    ## Creating the convolutional layer
    layer = tf.nn.conv2d(input=input,
                     filter=weights,
                     strides=[1, 1, 1, 1],
                     padding='SAME')
 
    layer += biases
 
    ## We shall be using max-pooling.  
    layer = tf.nn.max_pool(value=layer,
                            ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1],
                            padding='SAME')
    ## Output of pooling is fed to Relu which is the activation function for us.
    layer = tf.nn.relu(layer)
 
    return layer

def create_flatten_layer(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer = tf.reshape(layer, [-1, num_features])
 
    return layer

def create_fc_layer(input,          
             num_inputs,    
             num_outputs,
             use_relu=True):
    
    #Let's define trainable weights and biases.
    weights = create_weights(shape=[num_inputs, num_outputs])
    biases = create_biases(num_outputs)

    layer = tf.matmul(input, weights) + biases
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer

In [37]:
batch_size = 2
num_classes = 10
epochs = 12
X, Y = ds.make_dataset_for_classifier(path_with_jsons='../jsons/')
statistic = hlp.y_statistic(path_with_jsons='../jsons/')
y_generator = hlp.y_transform(statistic=statistic, Y=Y)
Y = np.asarray([y for y in y_generator])
cutX = []
for x in X:
    cutX.append(x[:1000, :])
X = np.asarray(cutX) #TODO решить что делать с размерами так как сейчас размеры такие (N, ?, 13)
num_channels = 1

In [33]:
a = tf.truncated_normal([ 2, 1000, 13])
sess = tf.Session()
sess.run(tf.initialize_all_variables())
sess.run(tf.shape(a))

array([   2, 1000,   13])

In [38]:
x = tf.placeholder(tf.float32, shape=[None, X.shape[0],X.shape[1], num_channels], name='x')
 
y_true = tf.placeholder(tf.float32, shape=[None, Y.shape[1]], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)

In [39]:
layer_conv1 = create_convolutional_layer(input=x,
               num_input_channels=num_channels,
               conv_filter_size=filter_size_conv1,
               num_filters=num_filters_conv1)
 
layer_conv2 = create_convolutional_layer(input=layer_conv1,
               num_input_channels=num_filters_conv1,
               conv_filter_size=filter_size_conv2,
               num_filters=num_filters_conv2)
 
layer_conv3= create_convolutional_layer(input=layer_conv2,
               num_input_channels=num_filters_conv2,
               conv_filter_size=filter_size_conv3,
               num_filters=num_filters_conv3)
          
layer_flat = create_flatten_layer(layer_conv3)
 
layer_fc1 = create_fc_layer(input=layer_flat,
                     num_inputs=layer_flat.get_shape()[1:4].num_elements(),
                     num_outputs=fc_layer_size,
                     use_relu=True)
 
layer_fc2 = create_fc_layer(input=layer_fc1,
                     num_inputs=fc_layer_size,
                     num_outputs=num_classes,
                     use_relu=False)

NameError: name 'filter_size_conv1' is not defined